In [ ]:
import h5py
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from crosscorr.crosscorr import Waveform, Binary, Orbit
YRSID_SI = 31558149.8 # 31558149.763545603

In [ ]:
t = np.arange(0, 1.0*YRSID_SI, 5.0)
rng = np.random.default_rng(seed=23)
u_rand = rng.random(5)
A = 1e-22
f = 1e-3
fdot = 0.0
iota = np.pi/2 - np.abs(np.arccos(2*u_rand[0]-1) - np.pi/2) 
phi0 = 2*np.pi*u_rand[1]
psi = 2*np.pi*u_rand[2]
beta = np.arccos(2*u_rand[3]-1)
lam = 2*np.pi*u_rand[4]

parameters = ['Amplitude', 'Frequency', 'FrequencyDerivative', 'Inclination',
          'InitialPhase', 'Polarization', 'EclipticLatitude', 'EclipticLongitude']
cat = np.vstack((A, f, fdot, iota, phi0, psi, beta, lam))
cat_data = np.atleast_2d(cat.T.ravel().view([(n, float) for n in parameters])).T

columns = ['t', 'X', 'Y', 'Z']
noise_level = 1e-22
noise = np.vstack((t, rng.normal(loc=0, scale=noise_level, size=(3, len(t)))))
noise_data = np.atleast_2d(noise.T.ravel().view([(n, float) for n in columns])).T

In [ ]:
filename = 'LDC2_sangria_training_v2.h5'
data_file = Path.cwd().joinpath('data').joinpath(filename)
data = h5py.File(filepath.joinpath(filename))

def parse_data(dat):
    return dat.view((float, len(dat.dtype.names))).T

sky_data = data['sky']
sim_signal = (parse_data(sky_data['igb']['tdi'][:, 0]) + 
              parse_data(sky_data['dgb']['tdi'][:, 0]) + 
              parse_data(sky_data['vgb']['tdi'][:, 0]) + 
              parse_data(sky_data['mbhb']['tdi'][:, 0]))

In [ ]:
full_signal = parse_data(data['obs']['tdi'][:, 0])
noise_signal = full_signal - sim_signal
time = full_signal[0, :]

plt.figure()
plt.plot(time[::50]/60/60/24, noise_signal[1, ::50], '#4c72b0', label='X', alpha=0.33)
plt.plot(time[::50]/60/60/24, noise_signal[2, ::50], '#c44e52', label='Y', alpha=0.33)
plt.plot(time[::50]/60/60/24, noise_signal[3, ::50], '#55a868', label='Z', alpha=0.33)
plt.legend()
plt.grid()
plt.show()

In [ ]:
L = 2.5e9
orbit = Orbit(t, L)
binary = Binary(A, iota, f, fdot, phi0, psi, beta, lam)
waveform = Waveform(t, binary, orbit)
tdi = np.vstack((t, np.real(waveform.X_tdi) + noise[1], 
                    np.real(waveform.Y_tdi) + noise[2], 
                    np.real(waveform.Z_tdi) + noise[3]))

columns = ['t', 'X', 'Y', 'Z']
tdi_data = np.atleast_2d(tdi.T.ravel().view([(n, float) for n in columns])).T

In [ ]:
filename = 'single_source_high_noise_test'
filepath = Path.cwd().parent.parent.joinpath('cwwd/data')
with h5py.File(filepath.joinpath(f'{filename}.hdf5'), 'w-') as f:
    sgb = f.create_group('/sky/sgb')
    sgb.create_dataset('tdi', data=tdi_data)
    sgb.create_dataset('cat', data=cat_data)
    sgb.create_dataset('noise', data=noise_data)

In [ ]:
h5py.File(filepath.joinpath(f'{filename}.hdf5'))['sky/sgb/tdi'][:, 0]